In [11]:
import mysql.connector
from mysql.connector import errorcode

# Database connection details
HOST = "localhost"  # Change to your MySQL server's IP or hostname
ROOT_USER = "root"   # Root user for database setup
ROOT_PASSWORD = "rishu123"  # Replace with your root password
NEW_DB = "AI_FinanceTrackerDB"
REMOTE_USER = "remote_user"  # Username for remote access
REMOTE_PASSWORD = "remote_password"  # Password for remote user
REMOTE_HOST = "%"  # Use '%' for any host, or specify an IP for limited access

# Establish connection to MySQL server
try:
    conn = mysql.connector.connect(host=HOST, user=ROOT_USER, password=ROOT_PASSWORD)
    cursor = conn.cursor()
    
    # Create the database
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {NEW_DB};")
    cursor.execute(f"USE {NEW_DB};")

    # Create tables
    table_statements = [
    """CREATE TABLE IF NOT EXISTS Users (
        user_id INT AUTO_INCREMENT PRIMARY KEY,
        first_name VARCHAR(50),
        last_name VARCHAR(50),
        email VARCHAR(100) UNIQUE NOT NULL,
        password_hash VARCHAR(255) NOT NULL,
        created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
        last_login DATETIME,
        gender ENUM('Male', 'Female', 'Other') NOT NULL
    );""",

    """CREATE TABLE IF NOT EXISTS Income_Source (
        source_id INT AUTO_INCREMENT PRIMARY KEY,
        source_name VARCHAR(100) UNIQUE NOT NULL
    );""",

    """CREATE TABLE IF NOT EXISTS Income (
        income_id INT AUTO_INCREMENT PRIMARY KEY,
        amount DECIMAL(10, 2) NOT NULL,
        date DATE NOT NULL,
        user_id INT,
        source_id INT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE,
        FOREIGN KEY (source_id) REFERENCES Income_Source(source_id) ON DELETE SET NULL
    );""",

    """CREATE TABLE IF NOT EXISTS Expense_Category (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        category_name VARCHAR(50) UNIQUE NOT NULL
    );""",

    """CREATE TABLE IF NOT EXISTS Expenses (
        expense_id INT AUTO_INCREMENT PRIMARY KEY,
        amount DECIMAL(10, 2) NOT NULL,
        date DATE NOT NULL,
        user_id INT,
        category_id INT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES Expense_Category(category_id) ON DELETE SET NULL
    );""",

    """CREATE TABLE IF NOT EXISTS Frequency (
        frequency_id INT AUTO_INCREMENT PRIMARY KEY,
        frequency_name VARCHAR(50) UNIQUE NOT NULL
    );""",

    """CREATE TABLE IF NOT EXISTS Recurring_Expenses (
        recurring_id INT AUTO_INCREMENT PRIMARY KEY,
        amount DECIMAL(10, 2) NOT NULL,
        start_date DATE NOT NULL,
        end_date DATE,
        user_id INT,
        frequency_id INT,
        category_id INT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE,
        FOREIGN KEY (frequency_id) REFERENCES Frequency(frequency_id) ON DELETE SET NULL,
        FOREIGN KEY (category_id) REFERENCES Expense_Category(category_id) ON DELETE SET NULL
    );""",

    """CREATE TABLE IF NOT EXISTS Budget (
        budget_id INT AUTO_INCREMENT PRIMARY KEY,
        monthly_budget DECIMAL(10, 2) NOT NULL,
        last_defined_date DATE NOT NULL,
        user_id INT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE
    );""",

    """CREATE TABLE IF NOT EXISTS Financial_Goal (
        goal_id INT AUTO_INCREMENT PRIMARY KEY,
        goal_amount DECIMAL(10, 2) NOT NULL,
        goal_deadline DATE NOT NULL,
        budget_id INT,
        FOREIGN KEY (budget_id) REFERENCES Budget(budget_id) ON DELETE CASCADE
    );"""
]

    for statement in table_statements:
        cursor.execute(statement)

    print("Database and tables created successfully.")

    # Create remote user and grant privileges
    cursor.execute(f"CREATE USER IF NOT EXISTS '{REMOTE_USER}'@'{REMOTE_HOST}' IDENTIFIED BY '{REMOTE_PASSWORD}';")
    cursor.execute(f"GRANT ALL PRIVILEGES ON {NEW_DB}.* TO '{REMOTE_USER}'@'{REMOTE_HOST}';")
    cursor.execute("FLUSH PRIVILEGES;")

    print(f"Remote user '{REMOTE_USER}' created and granted access to database '{NEW_DB}'.")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Access denied. Check your username and password.")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist.")
    else:
        print(err)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


Database and tables created successfully.
Remote user 'remote_user' created and granted access to database 'AI_FinanceTrackerDB'.


In [17]:
import mysql.connector
from mysql.connector import errorcode
import random
import faker
from datetime import datetime, timedelta

# Database connection details
HOST = "localhost"
ROOT_USER = "root"
ROOT_PASSWORD = "rishu123"
NEW_DB = "AI_FinanceTrackerDB"

# Initialize Faker for realistic data generation
fake = faker.Faker("en_IN")

# Connect to the database
try:
    conn = mysql.connector.connect(host=HOST, user=ROOT_USER, password=ROOT_PASSWORD, database=NEW_DB)
    cursor = conn.cursor()

    print("Populating Users table...")

    # Track generated emails to ensure uniqueness
    generated_emails = set()

    # Populate Users table
    for _ in range(1000):
        first_name = fake.first_name()
        last_name = fake.last_name()

        # Ensure unique email generation
        email = None
        while not email or email in generated_emails:
            email = f"{first_name.lower()}{random.randint(1, 100)}@example.org"
        generated_emails.add(email)

        password_hash = fake.sha256()
        created_at = fake.date_time_this_decade()
        last_login = created_at + timedelta(days=random.randint(1, 365))

        try:
            cursor.execute(
                """INSERT INTO Users (first_name, last_name, email, password_hash, created_at, last_login)
                VALUES (%s, %s, %s, %s, %s, %s)""",
                (first_name, last_name, email, password_hash, created_at, last_login),
            )
        except mysql.connector.Error as err:
            print(f"Error inserting user: {err}")

    conn.commit()
    print("Users table populated successfully.")

    print("Populating Income_Source table...")

    # Predefined income sources
    income_sources = [
        "Salary",
        "Business",
        "Freelancing",
        "Investments",
        "Rental Income",
        "Gifts",
        "Other"
    ]

    for source in income_sources:
        try:
            cursor.execute(
                "INSERT IGNORE INTO Income_Source (source_name) VALUES (%s)", (source,)
            )
        except mysql.connector.Error as err:
            print(f"Error inserting income source: {err}")

    conn.commit()
    print("Income_Source table populated successfully.")

    print("Populating Income table...")

    # Populate Income table
    cursor.execute("SELECT user_id FROM Users")
    user_ids = [row[0] for row in cursor.fetchall()]

    cursor.execute("SELECT source_id FROM Income_Source")
    source_ids = [row[0] for row in cursor.fetchall()]

    for user_id in user_ids:
        for _ in range(random.randint(1, 7)):
            amount = round(random.uniform(1000, 100000), 2)
            date = fake.date_this_year()
            source_id = random.choice(source_ids)

            try:
                cursor.execute(
                    """INSERT INTO Income (amount, date, user_id, source_id)
                    VALUES (%s, %s, %s, %s)""",
                    (amount, date, user_id, source_id),
                )
            except mysql.connector.Error as err:
                print(f"Error inserting income: {err}")

    conn.commit()
    print("Income table populated successfully.")

    print("Populating Expense_Category table...")

    # Predefined expense categories
    expense_categories = [
        "Food",
        "Transport",
        "Housing",
        "Utilities",
        "Entertainment",
        "Healthcare",
        "Education",
        "Miscellaneous"
    ]

    for category in expense_categories:
        try:
            cursor.execute(
                "INSERT IGNORE INTO Expense_Category (category_name) VALUES (%s)", (category,)
            )
        except mysql.connector.Error as err:
            print(f"Error inserting expense category: {err}")

    conn.commit()
    print("Expense_Category table populated successfully.")

    print("Populating Expenses table...")

    # Populate Expenses table
    cursor.execute("SELECT category_id FROM Expense_Category")
    category_ids = [row[0] for row in cursor.fetchall()]

    for user_id in user_ids:
        for _ in range(random.randint(5, 25)):
            amount = round(random.uniform(100, 5000), 2)
            date = fake.date_this_year()
            category_id = random.choice(category_ids)

            try:
                cursor.execute(
                    """INSERT INTO Expenses (amount, date, user_id, category_id)
                    VALUES (%s, %s, %s, %s)""",
                    (amount, date, user_id, category_id),
                )
            except mysql.connector.Error as err:
                print(f"Error inserting expense: {err}")

    conn.commit()
    print("Expenses table populated successfully.")

    print("Populating Budget table...")

    # Populate Budget table
    for user_id in user_ids:
        monthly_budget = round(random.uniform(10000, 100000), 2)
        last_defined_date = fake.date_this_year()

        try:
            cursor.execute(
                """INSERT INTO Budget (monthly_budget, last_defined_date, user_id)
                VALUES (%s, %s, %s)""",
                (monthly_budget, last_defined_date, user_id),
            )
        except mysql.connector.Error as err:
            print(f"Error inserting budget: {err}")

    conn.commit()
    print("Budget table populated successfully.")

    print("Populating Financial_Goal table...")

    # Populate Financial_Goal table
    cursor.execute("SELECT budget_id FROM Budget")
    budget_ids = [row[0] for row in cursor.fetchall()]

    for budget_id in budget_ids:
        for _ in range(random.randint(1, 3)):
            goal_amount = round(random.uniform(5000, 500000), 2)
            goal_deadline = fake.date_this_year()

            try:
                cursor.execute(
                    """INSERT INTO Financial_Goal (goal_amount, goal_deadline, budget_id)
                    VALUES (%s, %s, %s)""",
                    (goal_amount, goal_deadline, budget_id),
                )
            except mysql.connector.Error as err:
                print(f"Error inserting financial goal: {err}")

    conn.commit()
    print("Financial_Goal table populated successfully.")

except mysql.connector.Error as err:
    print(f"Error populating database: {err}")

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


Populating Users table...
Error inserting user: 1062 (23000): Duplicate entry 'anita16@example.org' for key 'users.email'
Error inserting user: 1062 (23000): Duplicate entry 'jacob11@example.org' for key 'users.email'
Users table populated successfully.
Populating Income_Source table...
Income_Source table populated successfully.
Populating Income table...
Income table populated successfully.
Populating Expense_Category table...
Expense_Category table populated successfully.
Populating Expenses table...
Expenses table populated successfully.
Populating Budget table...
Budget table populated successfully.
Populating Financial_Goal table...
Financial_Goal table populated successfully.
